In [1]:
!python -m pip install nltk

In [2]:
# Import all requirments
import tensorflow as tf
# import tensorflow_datasets as tfds
import time
import  os
import io
# from nltk.translate.bleu_score import corpus_bleu
from sklearn.model_selection import train_test_split

In [3]:
start_time = time.time()
## load data
german_data_path = os.path.join(os.path.curdir,"./data/europarl-v7.de-en.de")
# german_data_path= os.path.join()"data/europarl-v7.de-en.de"
english_data_path = os.path.join(os.path.curdir,"./data/europarl-v7.de-en.en")
# english_data_path =  os.path.join(os.path.curdir,"data/europarl-v7.de-en.en")
print("Time required to load data --- %s seconds ---" % (time.time() - start_time))



Time required to load data --- 0.0009963512420654297 seconds ---


In [4]:
## Load data from file and into a dataset in memory
start_time = time.time()
def create_dataset(germ_path, eng_path):
  germ_lines = io.open(germ_path, encoding='UTF-8').read().strip().split('\n')
  eng_lines = io.open(eng_path, encoding='UTF-8').read().strip().split('\n')
  return germ_lines, eng_lines
german_data, english_data = create_dataset(german_data_path, english_data_path)
print("Time required to create dataset --- %s seconds ---" % (time.time() - start_time))



Time required to create dataset --- 7.79448676109314 seconds ---


In [5]:
print(len(german_data))
# print(german_data.shape)

# german_train_data = german_data[int(0.8*len(german_data)):]
# german_test_data = german_data[:int(0.2*len(german_data))]

german_train_data = german_data[int(.3*len(german_data)):]
german_test_data = german_data[:int(.1*len(german_data))]

# english_data_test = english_data[int(.5*len(english_data)):]

english_train_data = english_data[int(.3*len(english_data)):]
english_test_data = english_data[:int(.1*len(english_data))]

print(len(german_train_data))
print(len(german_test_data))
print(len(german_train_data)+len(german_test_data))

1920209
1344147
192020
1536167


In [6]:
## Tokenize german and english sequences
start_time = time.time()

en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
## Uncomment these lines for first time running the program to configure tokenizer (helps speed up run time)
# en_tokenizer.fit_on_texts(english_data)
# english_config = en_tokenizer.to_json()

english_config_path = os.path.join(r"./tokenizerConfig/english_config.json")

# english_config_path = r"C:\Users\justi\Documents\RDPTranslation\initalTest\tokenizerConfig\english_config.json"
## Uncomment these lines for first time running the program to configure tokenizer (helps speed up run time)
# io.open(english_config_path,'w').write(english_config)
# print("Wrote english config to ", english_config_path)

## Comment lines out first time runnint to avoid reloading config
english_config = io.open(english_config_path, encoding='UTF-8').read()
en_tokenizer =  tf.keras.preprocessing.text.tokenizer_from_json(english_config)
print("Read english tokenizer input from ", english_config_path)

## Tokenize english data
data_en = en_tokenizer.texts_to_sequences(english_train_data)
data_en = tf.keras.preprocessing.sequence.pad_sequences(data_en,padding='post')

data_en_test = en_tokenizer.texts_to_sequences(english_test_data)
data_en_test = tf.keras.preprocessing.sequence.pad_sequences(data_en_test,padding='post')


ge_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')

## Uncomment these lines for first time running the program to configure tokenizer (helps speed up run time)
# ge_tokenizer.fit_on_texts(german_data)
# german_config = ge_tokenizer.to_json()

german_config_path = os.path.join(r"./tokenizerConfig/german_config.json")
# german_config_path = r"C:\Users\justi\Documents\RDPTranslation\initalTest\tokenizerConfig\english_config.json"
## Uncomment these lines for first time running the program to configure tokenizer (helps speed up run time)
# io.open(german_config_path,'w').write(german_config)
# print("Wrote german config to ", german_config_path)

## Comment lines out first time runnint to avoid reloading config
german_config = io.open(german_config_path, encoding='UTF-8').read()
ge_tokenizer =  tf.keras.preprocessing.text.tokenizer_from_json(german_config)
print("Read german tokenizer input from ", german_config_path)

## tokenize german data
data_ge = ge_tokenizer.texts_to_sequences(german_train_data)
data_ge = tf.keras.preprocessing.sequence.pad_sequences(data_ge,padding='post',maxlen=668)


data_ge_test = ge_tokenizer.texts_to_sequences(german_test_data)
data_ge_test = tf.keras.preprocessing.sequence.pad_sequences(data_ge_test,padding='post',maxlen=256)

print("Time required to tokenize data --- %s seconds ---" % (time.time() - start_time))


Read english tokenizer input from  ./tokenizerConfig/english_config.json
Read german tokenizer input from  ./tokenizerConfig/german_config.json
Time required to tokenize data --- 92.45083594322205 seconds ---


In [7]:
data_ge_test = ge_tokenizer.texts_to_sequences(german_test_data)
data_ge_test = tf.keras.preprocessing.sequence.pad_sequences(data_ge_test,padding='post',maxlen=256)
print("retokenized german test")
Y_test = data_ge_test
Y_test = tf.reshape(Y_test, [Y_test.shape[0], Y_test.shape[1], 1])
print(Y_test.shape)

retokenized german test
(192020, 256, 1)


In [8]:

start_time = time.time()
## slice data with an 80/20 split
# X_train,  X_test, Y_train, Y_test = train_test_split(data_en,data_ge,test_size=0.4)
X_train,  X_test, Y_train, Y_test = data_en,data_en_test, data_ge, data_ge_test

print("Time required to slice data --- %s seconds ---" % (time.time() - start_time))


Time required to slice data --- 0.0 seconds ---


In [9]:
print(len(data_en))
print(len(data_ge))

1344147
1344147


In [10]:
## Convert data to tensor and print shape of X_train (english training data)
Dtype = tf.float32

X_train = tf.convert_to_tensor(X_train,dtype=Dtype)
X_train = tf.reshape(X_train, [X_train.shape[0], X_train.shape[1], 1])
print(X_train.shape)
X_test = tf.convert_to_tensor(X_test,dtype=Dtype)
X_test = tf.reshape(X_test, [X_test.shape[0], X_test.shape[1], 1])
print(X_test.shape)
Y_train = tf.convert_to_tensor(Y_train,dtype=Dtype)
Y_train = tf.reshape(Y_train, [Y_train.shape[0], Y_train.shape[1], 1])
print(Y_train.shape)
Y_test = tf.convert_to_tensor(Y_test,dtype=Dtype)
Y_test = tf.reshape(Y_test, [Y_test.shape[0], Y_test.shape[1], 1])
print(Y_test.shape)




(1344147, 668, 1)
(192020, 256, 1)


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Reshape: Dst tensor is not initialized. [Op:Reshape]

In [11]:


## Define model (cnn seq2seq)

#Hyperparams            (from https://www.tensorflow.org/addons/tutorials/networks_seq2seq_nmt)
BATCH_SIZE = 64
BUFFER_SIZE = 64 #len(X_train)
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dims = 256
rnn_units = 1024
dense_units = 1024
Dtype = tf.float32

# input_vocab_size = len(en_tokenizer.word_index)+1
# output_vocab_size = len(ge_tokenizer.word_index)+ 1
dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
example_X, example_Y = next(iter(dataset))
example_X = tf.reshape(example_X,[example_X.shape[0],example_X.shape[1],1])
example_Y = tf.reshape(example_Y,[example_Y.shape[0],example_Y.shape[1],1])

# example_X.set_shape([example_X.shape[0],example_X.shape[1],1])
print(example_X.shape)
print(example_Y.shape)

(64, 668, 1)
(64, 668, 1)


In [12]:
def buildModel():
    model = tf.keras.Sequential(layers=[
        # tf.keras.layers.LSTM(64),
        # tf.keras.layers.LSTM(16),
        tf.keras.layers.Conv1D(668,kernel_size=1),
        tf.keras.layers.Conv1D(128,kernel_size=1),
        tf.keras.layers.Conv1D(64,kernel_size=1),
        tf.keras.layers.Conv1D(1,kernel_size=1),
        tf.keras.layers.Flatten(),
        # tf.keras.layers.Conv1D(64,kernel_size=1), #input_shape=(64,668)
        # tf.keras.layers.MaxPool1D(),
        # tf.keras.layers.Conv1D(16,kernel_size=1),
        # tf.keras.layers.Dense(16),
        # # # tf.keras.layers.MaxPool1D(),
        # #
#         tf.keras.layers.Dense(16),
#         tf.keras.layers.Dense(128),
#         tf.keras.layers.Dense(668),

        # tf.keras.layers.Conv1DTranspose(1,kernel_size=1),
        # tf.keras.layers.Conv1DTranspose(668,kernel_size=1)

        ])
    return model


In [13]:
# def buildModel2():
#     model = tf.keras.Sequential(layers=[
#         # tf.keras.layers.LSTM(64),
#         # tf.keras.layers.LSTM(16),
#         # tf.keras.layers.Conv1D(668,kernel_size=1),
#         # tf.keras.layers.Conv1D(128,kernel_size=1),
#         # tf.keras.layers.Conv1D(64,kernel_size=1),
#         # tf.keras.layers.Conv1D(1,kernel_size=1),
#         tf.keras.layers.Flatten(),
#         # tf.keras.layers.Conv1D(64,kernel_size=1), #input_shape=(64,668)
#         # tf.keras.layers.MaxPool1D(),
#         # tf.keras.layers.Conv1D(16,kernel_size=1),
#         # tf.keras.layers.Dense(16),
#         # # # tf.keras.layers.MaxPool1D(),
#         # #
#         tf.keras.layers.Dense(668),
#         tf.keras.layers.Dense(668),
#         tf.keras.layers.Dense(668),
#         tf.keras.layers.Dense(668),
#         tf.keras.layers.Dense(668),
#         tf.keras.layers.Dense(668),
#         # tf.keras.layers.Conv1DTranspose(1,kernel_size=1),
#         # tf.keras.layers.Conv1DTranspose(668,kernel_size=1)

#         ])
#     return model


In [14]:
# start_time = time.time()
# densemodel = buildModel2()
# print("Time required to create model --- %s seconds ---" % (time.time() - start_time))

# # seq2seq.summary()

# ## Train model and print results
# densemodel.compile(optimizer= 'adam',loss="CategoricalCrossentropy")
# # seq2seq.build(input_shape=(668,64))
# # seq2seq.summary()
# densemodel.fit(x=example_X,y=example_Y,batch_size=BATCH_SIZE,epochs=600)
# densemodel.summary()
# densemodel.save(filepath="./models/dense")

In [15]:
start_time = time.time()
seq2seq = buildModel()
print("Time required to create model --- %s seconds ---" % (time.time() - start_time))

# seq2seq.summary()

## Train model and print results
seq2seq.compile(optimizer= 'adam',loss="mean_squared_logarithmic_error")
# seq2seq.build(input_shape=(668,64))
# seq2seq.summary()
seq2seq.fit(x=example_X,y=example_Y,batch_size=BATCH_SIZE,epochs=100)
seq2seq.summary()
seq2seq.save(filepath="./models/seq2seq")


Time required to create model --- 0.02201056480407715 seconds ---
Epoch 1/100
1/1 [==============================] - 0s 2ms/step - loss: 1.2541
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 1.2541
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 1.2541
Epoch 4/100
1/1 [==============================] - 0s 988us/step - loss: 1.2541
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 1.2541
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 1.2541
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 1.2541
Epoch 8/100
1/1 [==============================] - 0s 999us/step - loss: 1.2541
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 1.2541
Epoch 10/100
1/1 [==============================] - 0s 1ms/step - loss: 1.2541
Epoch 11/100
1/1 [==============================] - 0s 2ms/step - loss: 1.2541
Epoch 12/100
1/1 [==============================] - 0s 1ms/step - loss

In [16]:
seq2seq.evaluate(X_test,  Y_test, verbose=2)

6001/6001 - 13s - loss: 3.3357


3.3357207775115967

In [17]:
print(X_test.shape)
print(Y_test.shape)

(192020, 256, 1)
(192020, 256)


In [18]:
seq2seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              multiple                  1336      
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  85632     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  8256      
_________________________________________________________________
conv1d_3 (Conv1D)            multiple                  65        
_________________________________________________________________
flatten (Flatten)            multiple                  0         
Total params: 95,289
Trainable params: 95,289
Non-trainable params: 0
_________________________________________________________________


In [19]:
output = seq2seq.predict(X_test)

In [20]:
print(output)

[[-9.4237805e+02 -3.2784075e-01 -1.6392037e-01 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-1.8031214e+00 -4.7356549e+02 -5.4831317e+02 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-1.1402291e+03 -2.7866426e+00 -6.8846436e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 ...
 [-1.6392032e+00 -2.9505656e+00 -7.4255905e+01 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-1.8031214e+00 -2.0294944e+03 -2.9833488e+01 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-1.8031214e+00 -4.2455307e+01 -1.6392037e-01 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]


In [21]:
print(output.shape)

(192020, 256)


In [22]:
german_output = ge_tokenizer.sequences_to_texts(output)

In [23]:
print(german_output[0])

In [24]:
print(german_output[5:])

'', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 

In [25]:
clean_output = german_output                    

for i in clean_output:
    clean_output.remove('')
        

In [26]:
print(len(german_output))
print(len(clean_output))

96010
96010


In [27]:
for i in range(0,10):
    print(clean_output[i])

In [28]:
print(clean_output[5:])

'', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 

In [29]:
string_output = " "
string_output = string_output.join(clean_output)

In [30]:
print(string_output)

In [31]:
test = data_ge_test
print(test.shape)
german_controll = ge_tokenizer.sequences_to_texts(test)
for i in range(0,10):
    print(german_controll[i])

(192020, 256)
wiederaufnahme der sitzungsperiode
ich erkläre die am freitag, dem 17. dezember unterbrochene sitzungsperiode des europäischen parlaments für wiederaufgenommen, wünsche ihnen nochmals alles gute zum jahreswechsel und hoffe, daß sie schöne ferien hatten.
wie sie feststellen konnten, ist der gefürchtete "millenium-bug " nicht eingetreten. doch sind bürger einiger unserer mitgliedstaaten opfer von schrecklichen naturkatastrophen geworden.
im parlament besteht der wunsch nach einer aussprache im verlauf dieser sitzungsperiode in den nächsten tagen.
heute möchte ich sie bitten - das ist auch der wunsch einiger kolleginnen und kollegen -, allen opfern der stürme, insbesondere in den verschiedenen ländern der europäischen union, in einer schweigeminute zu gedenken.
ich bitte sie, sich zu einer schweigeminute zu erheben.
(das parlament erhebt sich zu einer schweigeminute.)
frau präsidentin, zur geschäftsordnung.
wie sie sicher aus der presse und dem fernsehen wissen, gab es in sr

In [32]:
english_test = en_tokenizer.sequences_to_texts(output)
for i in range(0,10):
    print(english_test[i])

In [33]:
print(english_test[5:])

'', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 

In [34]:
english_controll_output = " "
english_controll_output = english_controll_output.join(english_test)

In [35]:
print(english_controll_output)